# Function definitions

In [1]:
def matrix_multiply(A, B):
    n = len(A)
    p = len(B[0])
    m = len(B)
    result = [[0] * p for _ in range(n)]
    for i in range(n):
        for j in range(p):
            for k in range(m):
                result[i][j] += A[i][k] * B[k][j]
    return result


In [2]:
def matrix_multiply_np(A, B,Reps):
    C=np.dot(A, B)
    for i in range(Reps):
        C=np.dot(A, C)
    return C    


In [4]:
def matrix_multiply_torch(A, B, Reps):
    # Check if CUDA (GPU support) is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    A_torch = torch.tensor(A, dtype=torch.float64).to(device)
    B_torch = torch.tensor(B, dtype=torch.float64).to(device)
    C_torch = torch.matmul(A_torch, B_torch)  # Perform the first multiplication on the GPU

    for i in range(Reps):  # Subtract 1 because the first multiplication is already done
        C_torch = torch.matmul(A_torch, C_torch)  # Further multiplications on the GPU

    C = C_torch.cpu().numpy()  # Only move the final result to CPU and convert to NumPy
    return C


# Benchmarking functions

In [5]:
import time
import numpy as np
import torch

# Generate random matrices
n = 512
Reps=20
A = np.random.rand(n, n).astype(np.float64)
B = np.random.rand(n, n).astype(np.float64)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is:",device)

#Measure performance of plain Python
start_time = time.time()
result_python = matrix_multiply(A.tolist(), B.tolist())
print("Plain Python time:", time.time() - start_time)

# Measure performance of NumPy
start_time = time.time()
result_np = matrix_multiply_np(A, B,Reps)
print("NumPy time:", time.time() - start_time)

# Measure performance of PyTorch with GPU
start_time = time.time()
result_torch = matrix_multiply_torch(A, B,Reps)
print("PyTorch (GPU) time:", time.time() - start_time)



device is: cuda
Plain Python time: 34.920814514160156
NumPy time: 0.16881370544433594
PyTorch (GPU) time: 0.35549473762512207


# Using timeit to properly benchmark functions

In [6]:
import matplotlib
matplotlib.use('Agg')  # Use the 'Agg' backend for file creation
import torch
import timeit
import numpy as np
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is:",device)

n = 512
A = np.random.rand(n, n).astype(np.float64)
B = np.random.rand(n, n).astype(np.float64)
reps_range = range(10, 61, 10)
numpy_times = []
torch_times = []
number = 40

for Reps in reps_range:
    # Time NumPy
    numpy_time = timeit.timeit('matrix_multiply_np(A, B, Reps)', globals=globals(), number=number)/number
    print('numpy ',Reps, numpy_time)
    numpy_times.append(numpy_time)

    # Time PyTorch
    torch_time = timeit.timeit('matrix_multiply_torch(A, B,  Reps)', globals=globals(), number=number)/number
    print('Torch ',Reps, torch_time)
    torch_times.append(torch_time)

plt.figure(figsize=(10, 5))
plt.plot(reps_range, numpy_times, label='NumPy', marker='o')
plt.plot(reps_range, torch_times, label='PyTorch', marker='x')
plt.xlabel('Number of Repetitions (Reps)')
plt.ylabel('Time (seconds)')
plt.title('Performance Comparison: NumPy vs. PyTorch GPU')
plt.legend()
plt.grid(True)
plt.draw()
plt.savefig('reps_docker.pdf')


device is: cuda
numpy  10 0.17949371734994202
Torch  10 0.029094928074846392
numpy  20 0.16577136422511102
Torch  20 0.045954912174966014
numpy  30 0.2015374072751001
Torch  30 0.06438540879989887
numpy  40 0.5141295618001095
Torch  40 0.08124777332486702
numpy  50 0.36966224070001774
Torch  50 0.09956418292495073
numpy  60 0.5264189494249877
Torch  60 0.11781185972504318


# Using more GPU intensive calculations

In [7]:
import numpy as np
import torch
import time
import matplotlib.pyplot as plt

def multiply_numpy(A, B):
    return np.dot(A, B)

def multiply_torch(A, B):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    A_torch = torch.tensor(A, dtype=torch.float32).to(device)
    B_torch = torch.tensor(B, dtype=torch.float32).to(device)
    return torch.matmul(A_torch, B_torch).cpu().numpy()

# Sizes of the matrices to test
sizes = [256, 512, 2**10, 2**11, 2**12,2**13]
numpy_times = []
torch_times = []


for size in sizes:
    A = np.random.rand(size, size).astype(np.float64)
    B = np.random.rand(size, size).astype(np.float64)

    # Time NumPy
    start_time = time.time()
    Cnp=multiply_numpy(A, B)
    numpy_duration = time.time() - start_time
    numpy_times.append(numpy_duration)

    # Time PyTorch with GPU
    start_time = time.time()
    Ctorch=multiply_torch(A, B)
    torch_duration = time.time() - start_time
    torch_times.append(torch_duration)

    print(f"Size: {size}x{size}")
    print("NumPy Duration:", numpy_duration)
    print("Torch GPU Duration:", torch_duration)
    print("Difference:",np.max(np.abs(Cnp-Ctorch)),'\n')    
    

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(sizes, numpy_times, label='NumPy', marker='o')
plt.plot(sizes, torch_times, label='PyTorch GPU', marker='x')
plt.xlabel('Matrix Size')
plt.ylabel('Time (seconds)')
plt.title('Performance Comparison: NumPy vs. PyTorch GPU')
plt.yscale('log')  # Set the y-axis to logarithmic scale
plt.legend()
plt.grid(True, which="both", ls="--")  # Enable grid for major and minor ticks
plt.draw()
plt.savefig('sizes_docker.pdf')


Size: 256x256
NumPy Duration: 0.007596492767333984
Torch GPU Duration: 0.0424041748046875
Difference: 1.952616858602596e-05 

Size: 512x512
NumPy Duration: 0.022054433822631836
Torch GPU Duration: 0.05364179611206055
Difference: 8.04747834877162e-05 

Size: 1024x1024
NumPy Duration: 0.054381608963012695
Torch GPU Duration: 0.07471203804016113
Difference: 0.00020822434919409716 

Size: 2048x2048
NumPy Duration: 0.5141119956970215
Torch GPU Duration: 0.14284086227416992
Difference: 0.0006645154892339633 

Size: 4096x4096
NumPy Duration: 3.4417924880981445
Torch GPU Duration: 0.34295654296875
Difference: 0.004599885652623925 

Size: 8192x8192
NumPy Duration: 15.62960433959961
Torch GPU Duration: 0.9709539413452148
Difference: 0.014186640691605135 



# Different GPU optimized functions

In [8]:
import numpy as np
import torch
from PIL import Image
import timeit

# Example image loading and conversion to numpy
image = Image.open('Kinkade.jpg').convert('RGB')  # Ensure image is in RGB
image_np = np.array(image).astype(np.float32)  # Convert to float for better handling in PyTorch
image_torch = torch.tensor(image_np.transpose(2, 0, 1)).unsqueeze(0).to('cuda')  # Convert to Tensor and move to GPU

# Define the target size as a tuple (width, height)
size = (512*15, 512*15)  # Define the size as a tuple

def numpy_resize(image_np, size):
    return np.array(Image.fromarray(image_np.astype(np.uint8)).resize(size))

def torch_resize(image_torch, size):
    # size needs to be (height, width) for PyTorch, reverse the tuple
    torch_size = (size[1], size[0])
    return torch.nn.functional.interpolate(image_torch, size=torch_size, mode='bilinear', align_corners=False).cpu()

# Timing the functions using timeit
number = 10  # Number of iterations for timing

# Time NumPy
numpy_time = timeit.timeit('numpy_resize(image_np, size)', globals=globals(), number=number) / number

# Time PyTorch
torch_time = timeit.timeit('torch_resize(image_torch, size)', globals=globals(), number=number) / number

print("NumPy Duration:", numpy_time)
print("Torch GPU Duration:", torch_time)


NumPy Duration: 1.2434653197000443
Torch GPU Duration: 0.705101901099988


# Functions not optimized for GPU

In [9]:
# Creating a large array and summing it
import timeit
import numpy as np
import torch
large_data = np.random.rand(10**8)  # 10 million elements


def sum_numpy(data):
    return np.sum(data)

def sum_torch(data):
    tensor = torch.tensor(data).cuda()
    return torch.sum(tensor).cpu()

number=10
# Timing the sum operations
numpy_time = timeit.timeit('sum_numpy(large_data)', globals=globals(), number=number) / number


torch_time = timeit.timeit('sum_torch(large_data)', globals=globals(), number=number) / number


print("NumPy time:", numpy_time)
print("Torch time:", torch_time)


NumPy time: 0.17585229750038706
Torch time: 0.5166345161000209
